In [153]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import DataLoader
import AlphaNet 
from AlphaNet.Models import AlphaNet_LSTM_V1

# read_task
Alpha_Name = "Original_Input_Shift_1_Sequence_10"
data_path = "/home/ShareFolder/feature_platform/ti0/wuwenjun/#Data_lib/"

# Train Loader
trainloader = DataLoader()
trainloader.load_data_from_file(alpha_name = Alpha_Name,start_date = "2018-03-01",end_date = "2018-03-03",data_path=data_path)
# train_loader = trainloader.to_torch_DataLoader(sequence = sequence,shuffle=False)

100%|██████████| 2/2 [00:00<00:00, 32.38it/s]


open_Shift_9  high_Shift_9  low_Shift_9  \
timestamp           ticker                                            
2018-03-01 09:30:00 000001       1363.94       1373.51      1332.05   
                    000002       4530.90       4651.73      4460.88   
                    000004         80.55         81.44        79.73   
                    000005         33.83         34.48        33.64   
                    000008        168.48        171.13       167.82   
...                                  ...           ...          ...   
2018-03-02 09:30:00 603991         21.37         21.50        20.00   
                    603993         21.27         21.72        20.01   
                    603996         24.27         24.66        24.14   
                    603997         16.46         16.46        15.79   
                    603999         16.40         16.40        15.92   

                            close_Shift_9  vwap_Shift_9  volume_Shift_9  \
timestamp           ticker                                                
2018-03-01 09:30:00 000001        1333.11   1348.346441      2137815.06   
                    000002        4496.58   4541.171859       580805.14   
                    000004          81.40     81.185397         5607.93   
                    000005          34.38     34.165018        21063.92   
                    000008         168.92    168.958750        37701.66   
...                                   ...           ...             ...   
2018-03-02 09:30:00 603991          20.65     20.932200         5027.00   
                    603993          20.01     20.534599      1977915.24   
                    603996          24.24     24.300927        15833.19   
                    603997          16.25     16.032886        39389.50   
                    603999          16.11     16.137726        21332.20   

                            pct_chg_Shift_9  turnover_Shift_9  \
timestamp           ticker                                      
2018-03-01 09:30:00 000001          -2.9412            1.2636   
                    000002          -0.6974            0.5982   
                    000004           0.6533            0.6755   
                    000005           1.6438            0.2214   
                    000008           0.3932            0.1875   
...                                     ...               ...   
2018-03-02 09:30:00 603991          -5.5784            2.6882   
                    603993         -10.0145            1.5269   
                    603996          -1.2477            1.5331   
                    603997          -0.9337            2.3035   
                    603999          -2.9155            0.9259   

                            free_turnover_Shift_9  open_Shift_8  ...  \
timestamp           ticker                                       ...   
2018-03-01 09:30:00 000001                 2.8091       1284.21  ...   
                    000002                 1.0483       4324.95  ...   
                    000004                 1.1372         79.85  ...   
                    000005                 0.2772         34.10  ...   
                    000008                 0.2543        168.26  ...   
...                                           ...           ...  ...   
2018-03-02 09:30:00 603991                 2.6882         20.76  ...   
                    603993                 8.6344         20.14  ...   
                    603996                 1.5331         24.27  ...   
                    603997                 2.3035         16.23  ...   
                    603999                 0.9259         16.16  ...   

                            open_Shift_0  high_Shift_0  low_Shift_0  \
timestamp           ticker                                            
2018-03-01 09:30:00 000001       1286.34       1295.90      1268.26   
                    000002       4463.62       4562.48      4371.63   
                    000004         83.31         83.72       

In [154]:
train_loader = trainloader.to_torch_DataLoader(sequence = 10,shuffle=False)

In [13]:
pd.DataFrame(train_loader.dataset[1][0].detach().numpy())

,0,1,2,3,4,5,6,7,8
0,4530.90,4651.73,4460.88,4496.58,4541.171859,580805.14,-0.6974,0.5982,1.0483
1,4324.95,4388.11,4146.46,4293.37,4278.475664,724944.90,-4.5191,0.7467,1.3085
2,4315.34,4437.54,4298.86,4379.87,4368.983860,407654.31,2.0147,0.4199,0.7358
3,4432.05,4639.37,4432.05,4493.83,4532.700446,454668.11,2.6019,0.4683,0.8207
4,4513.05,4574.84,4416.94,4554.24,4494.695790,270175.50,1.3443,0.2783,0.4877
5,4668.20,4675.07,4576.21,4627.01,4628.082794,302373.26,1.5978,0.3114,0.5458
6,4654.47,4819.23,4576.21,4767.06,4725.524643,384538.90,3.0267,0.3961,0.6941
7,4815.11,4819.23,4546.00,4692.92,4653.702984,540655.01,-1.5553,0.5569,0.9759
8,4695.66,4706.65,4482.85,4517.17,4561.808057,506598.14,-3.7449,0.5218,0.9144
9,4463.62,4562.48,4371.63,4489.71,4453.945134,500004.47,-0.6079,0.5150,0.9025


In [51]:
check = DataLoader()
check.load_data_from_file(alpha_name = "Original_Input",start_date = "2018-02-01",end_date = "2018-03-03",data_path=data_path)

100%|██████████| 17/17 [00:00<00:00, 89.24it/s]


open      high       low     close  \
timestamp           ticker                                           
2018-02-01 09:30:00 600651  57721.67  57780.87  54702.38  56004.82   
                    600652  39169.60  40529.66  38843.19  38843.19   
                    600653  26446.33  26446.33  24032.98  24334.65   
                    600601  18440.93  19426.76  18324.95  18440.93   
                    000651   8073.78   8456.53   8055.84   8363.84   
...                              ...       ...       ...       ...   
2018-03-02 09:30:00 002420      4.43      4.58      4.42      4.55   
                    000585      4.33      4.38      4.29      4.35   
                    601188      4.27      4.31      4.26      4.31   
                    601518      3.71      3.76      3.70      3.76   
                    601866      3.19      3.20      3.16      3.20   

                                    vwap     volume  pct_chg  turnover  \
timestamp           ticker                                               
2018-02-01 09:30:00 600651  56342.267378  144891.62  -3.7640    1.4707   
                    600652  39278.404277  269600.36  -9.9622    3.2377   
                    600653  24947.037479  244968.91  -8.3333    1.4027   
                    600601  18599.821600  412211.36  -9.4017    1.8780   
                    000651   8281.961229  667284.56   2.9255    1.1176   
...                                  ...        ...      ...       ...   
2018-03-02 09:30:00 002420      4.535685   20501.85   1.8223    0.5234   
                    000585      4.345992   47517.79   0.8876    0.7800   
                    601188      4.287910   11803.69   0.5038    0.0897   
                    601518      3.738836   22173.87   0.5952    0.1828   
                    601866      3.186363   85204.39   0.6349    0.1074   

                            free_turnover  
timestamp           ticker                 
2018-02-01 09:30:00 600651         2.4149  
                    600652         5.2142  
                    600653         1.6331  
                    600601         2.1256  
                    000651         1.5382  
...                                   ...  
2018-03-02 09:30:00 002420         0.7524  
                    000585         0.9005  
                    601188         0.2865  
                    601518         0.5196  
                    601866         0.2420  

[52243 rows x 9 columns]

In [160]:

x.shape

torch.Size([6182, 90])

In [146]:
class AlphaNet_LSTM_V1(nn.Module):
    def __init__(self, factor_num,sequence,fully_connect_layer_neural):
        super(AlphaNet_LSTM_V1, self).__init__()
        self.factor_num = factor_num  # 108
        self.sequence = sequence
        self.fc2_neuron = fully_connect_layer_neural  # 32

        # Layer
        self.batch = torch.nn.BatchNorm1d(self.factor_num * self.sequence)
        self.lstm = nn.LSTM(self.factor_num, self.fc2_neuron, 3, batch_first=True, bidirectional=True,dropout = 0.2)
        self.lstm2 = nn.LSTM(int(self.fc2_neuron *2), int(self.fc2_neuron/2), 3, batch_first=True, bidirectional=True,dropout = 0.2)
        self.batch2 = torch.nn.BatchNorm1d(self.fc2_neuron)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.LeakyReLU = nn.LeakyReLU()
        self.out = nn.Linear(self.fc2_neuron, 1)

    def forward(self, x):
        x = x.reshape(x.shape[0],-1).float()
        x = self.batch(x)
        x = x.reshape(x.shape[0],self.sequence,self.factor_num)
        r_out, _ = self.lstm(x)  # r_out.shape: torch.Size([6182, 10, 128])
        r_out = self.LeakyReLU(r_out) # torch.Size([6182, 10, 128])
        r_out, (hn, cn) = self.lstm2(r_out) # torch.Size([6182, 10, 64])
        r_out = r_out[:, -1] # torch.Size([6182, 64])
        r_out = self.batch2(r_out)  # torch.Size([6182, 64])
        r_out = self.relu(r_out)
        r_out = self.dropout(r_out)
        y_pred = self.out(r_out)
        return y_pred

In [177]:
# x = torch.from_numpy(np.array(trainloader.feature_data.drop("target",axis=1)))
# x = x.reshape(6182,10,9)
# x = x.reshape(x.shape[0],-1).float()
# r_out, (hn, cn) = nn.LSTM(factor_num, fc2_neuron, 3, batch_first=True, bidirectional=True,dropout = 0.2)(x)
# r_out, (hn, cn) = nn.LSTM(int(fc2_neuron *2), int(fc2_neuron/2), 3, batch_first=True, bidirectional=True,dropout = 0.2)(r_out)
# nn.Linear(fc2_neuron, 1)(r_out)
# x = torch.nn.BatchNorm1d(90)(x)
x[4]

tensor([ 0.0457,  0.0437,  0.0476,  0.0446,  0.0448, -0.2083,  0.3954, -0.5658,
        -0.6897,  0.0455,  0.0437,  0.0446,  0.0423,  0.0428, -0.1058, -0.2367,
        -0.5062, -0.6089,  0.0412,  0.0395,  0.0423,  0.0410,  0.0402, -0.2088,
        -0.1676, -0.5333, -0.6290,  0.0422,  0.0401,  0.0423,  0.0420,  0.0414,
        -0.2393,  0.3666, -0.5280, -0.6211,  0.0413,  0.0448,  0.0430,  0.0457,
         0.0444, -0.1700,  1.0606, -0.4891, -0.5762,  0.0451,  0.0470,  0.0469,
         0.0456,  0.0468, -0.1787, -0.1661, -0.4975, -0.5888,  0.0453,  0.0439,
         0.0437,  0.0414,  0.0426, -0.1986, -1.4501, -0.5441, -0.6487,  0.0416,
         0.0399,  0.0423,  0.0406,  0.0405, -0.1785, -0.3554, -0.5574, -0.6541,
         0.0407,  0.0378,  0.0369,  0.0384,  0.0368,  0.0427, -0.8362, -0.4498,
        -0.5203,  0.0363,  0.0331,  0.0350,  0.0335,  0.0333,  0.1598, -1.6431,
        -0.4689, -0.5541], grad_fn=<SelectBackward0>)

In [149]:
from torch.autograd import Variable
alphanet = AlphaNet_LSTM_V1(factor_num,sequence,64)
alphanet = alphanet.cuda()
print(alphanet)
LR = 0.01
loss_function = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(alphanet.parameters(), lr=LR)
epoch_num = 50
loss_list = []

min_loss = float("inf")
for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad()
        
        pred = alphanet(inputs)
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        loss.backward()
        optimizer.step()
        

        total_loss += loss.item()
    total_loss = total_loss * 1024 / 1
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

AlphaNet_LSTM_V1(
  (batch): BatchNorm1d(90, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(9, 64, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (lstm2): LSTM(128, 32, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (batch2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (LeakyReLU): LeakyReLU(negative_slope=0.01)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


  2%|▏         | 1/50 [00:02<02:17,  2.81s/it]

Epoch:  1  loss:  5740.770263671875


  4%|▍         | 2/50 [00:05<02:10,  2.72s/it]

Epoch:  2  loss:  5567.011413574219


Exception ignored in: <function _releaseLock at 0x7fdb5a1c5680>
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7fdb5a1c5680>
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7fdb5a1c5680>
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7fdb5a1c5680>
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7fdb5a1c5680>
Traceback (most 

KeyboardInterrupt: 

In [178]:
a = []
for i in range(20):
    a.append(i)

In [185]:
# a = torch.from_numpy((np.array(a)).reshape(1,20))
a.shape

torch.Size([1, 20])

In [187]:
a.reshape(2,5,2)[0]

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])